[last week's code](../w9/varCode.ipynb)

In [1]:
import pandas as pd

In [5]:
df=pd.read_csv('../w9/bw0803.csv')
len(df[df.IC2_sev==df.IC3_sev])

46

In [13]:
df[(df.RRincidents==0) & (df.IC1_date.isna()!=False)][df.columns[-28:]]

,doubt,handcoded,round,RRincidents,IC1_date,IC1_sev,IC1_reach,IC1_nov,IC1_env,IC1_soc,...,IC2_gov,IC2_cros,IC3_date,IC3_sev,IC3_reach,IC3_nov,IC3_env,IC3_soc,IC3_gov,IC3_cros
34,0.0,0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,0.0,0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,0.0,0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,0.0,0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,0.0,0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6304,0.0,0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6374,0.0,0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6375,0.0,0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6395,0.0,0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
def max_m(n: int, digits: set[int]) -> int:
    s_n = str(n)
    s_digits = sorted(map(str, digits), reverse=True)
    mp = dict()  # k : v  digits中小于k的最大值
    for k in '0123456789':
        mp[k] = '#'
        for v in s_digits:
            if v < k:
                mp[k] = v
                break
    possible_b = -1
    for i, c in enumerate(s_n):
        if mp[c] != '#':
            possible_b = i
        if c not in s_digits:
            break
    ret = s_digits[0] * (len(s_n) - 1)
    if possible_b != -1:
        ret = s_n[:possible_b] + mp[s_n[possible_b]] + s_digits[0] * (len(s_n) - possible_b - 1)
    return int(ret)


max_m(21000,[2,4,9])

22999

In [45]:
sorted(['aa','a'])

['a', 'aa']

In [46]:
def kthDistinct( arr, k: int) -> str:
        sarr=sorted(arr)
        dup=set([sarr[i] for i in range(len(sarr)-1) if sarr[i]==sarr[i+1]])
        
        if dup==set():
            return arr[k-1]
        else:
            try:
                return list(set(arr)-dup)[k-1]
            except IndexError:
                return ''


kthDistinct(["jqrd","k","miy","svuwg","uv","yttn","bxu","nymzu","dpane","x","mb","zm","ae","ihwtn","kouej","y","zt","h","udx","h","imi","zombs","l","hvt","uor","kzi","tzm","kde","ml","hmvz","hriuy","lav","hlvw","fnnj","bdkh","hu","tuuob","uc","no","qo","ku","foe"],27)

['hriuy',
 'ae',
 'uv',
 'qo',
 'hmvz',
 'ku',
 'no',
 'zm',
 'ihwtn',
 'nymzu',
 'zt',
 'fnnj',
 'lav',
 'ml',
 'hu',
 'jqrd',
 'udx',
 'foe',
 'uor',
 'kouej',
 'dpane',
 'l',
 'tuuob',
 'uc',
 'zombs',
 'bdkh',
 'mb',
 'svuwg',
 'x',
 'yttn',
 'imi',
 'k',
 'y',
 'hlvw',
 'kde',
 'tzm',
 'kzi',
 'miy',
 'bxu',
 'hvt']